In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import csv 
from sklearn.model_selection import train_test_split
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/USYD/2021 S1/INFO4001/deng-reads-RawCount-modefied.csv',header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

In [ ]:
def shuffle_data(x):
    
    # save header
    tmp_head = x[0]

    #remove header 
    x = x[1:]

    # shuffle by row 
    np.random.shuffle(x)

    #stack header
    x = np.vstack((tmp_head,x))

    # shuffle by column after T transformation 
    x = x.T 

    tmp_head = x[0]
    x = x[1:]

    np.random.shuffle(x)

    x = np.vstack((tmp_head,x))

    return x 

In [ ]:
# data preprocessing and data split 

mt = df.to_numpy()
#mt = mt.T

#shuffle dataset by rows and cols
mt = shuffle_data(mt)

# get type for each sample 
mt_Y = mt.T[0][1:]

mt1 = mt[:,1:]
mt_X = mt1[1:]
mt_X = mt_X.astype('float64')


unique_classes = np.unique(mt_Y)

# split data to train and test 
x_train, x_test, y_train, y_test = train_test_split(
    mt_X, mt_Y, test_size=0.2,stratify=mt_Y, random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(unique_classes)
print(len(unique_classes))

(214, 21297) (54, 21297) (214,) (54,)
['16cell' '2cell' '4cell' '8cell' 'blast' 'zygote']
6


In [ ]:
#Different normalisation method: raw, CPM, minmax(CPM)

#cpm only
tmp_train = np.nan_to_num(x_train)
tmp_test = np.nan_to_num(x_test)
sum_train = np.sum(tmp_train,axis=1)
sum_train = sum_train.reshape(sum_train.shape[0],-1)
sum_test = np.sum(tmp_test,axis=1)
sum_test = sum_test.reshape(sum_test.shape[0],-1)
#print(tmp_train.shape)
#print(np.sum(tmp_train,axis=1))

#cpm_x_train = np.divide(tmp_train, np.sum(tmp_train,axis=1))



#print(a2.shape)
#cpm_x_train = tmp_train /a2
cpm_x_train = np.divide(tmp_train, sum_train)
cpm_x_test =  np.divide(tmp_test, sum_test)
print(cpm_x_train.shape)



#cpm with minmax
mnx = MinMaxScaler()

mnx_x_train = mnx.fit_transform(cpm_x_train)

mnx_x_test = mnx.transform(cpm_x_test)



(214, 21297)


In [ ]:

label = unique_classes.tolist()


n_lst=[]
for i in range(len(y_train)):
  for j in range(len(label)):
    if y_train[i] == label[j]:

      n_lst.append(j)

y_new_train = np.array(n_lst,dtype='float64')

n_lst=[]
for i in range(len(y_test)):
  for j in range(len(label)):
    if y_test[i] == label[j]:
      n_lst.append(j)

y_new_test = np.array(n_lst,dtype='float64')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train,y_new_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = clf.predict(x_test)



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

out_acc = accuracy_score(y_new_test, y_pred)
print("Raw data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("Raw data F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("Raw data F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("Raw data F1 Score(weighted): {}".format(round(out_f1,4)))

Raw data Accuracy: 0.7593
Raw data F1 Score: 0.7593


In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(cpm_x_train,y_new_train)
y_pred = clf.predict(cpm_x_test)

out_acc = accuracy_score(y_new_test, y_pred)
print("CPM data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("CPM data  F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("CPM data  F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("CPM data  F1 Score(weighted): {}".format(round(out_f1,4)))


CPM data Accuracy: 0.7593
CPM data F1 Score: 0.7593


In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(mnx_x_train,y_new_train)
y_pred = clf.predict(mnx_x_test)

out_acc = accuracy_score(y_new_test, y_pred)
print("mixmax(CPM) data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("mixmax(CPM) data  F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("mixmax(CPM) data  F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("mixmax(CPM) data  F1 Score(weighted): {}".format(round(out_f1,4)))

minmax(CPM) data Accuracy: 0.7963
minmax(CPM) data F1 Score: 0.7963
